In [87]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
import tensorflow as tf

In [88]:
# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt
import collections
import random
import numpy as np
import os
import time
import json
from PIL import Image
import glob
import re
from PIL import Image

In [89]:
#última camada convolucional na arquitetura InceptionV3
#O formato da saída dessa camada é 8x8x2048
#encaminho cada imagem pela rede e armazena o vetor resultante em um dicionário (image_name -> feature_vector).
#Depois que todas as imagens são transmitidas pela rede, você salva o dicionário no disco.
image_model = tf.keras.applications.InceptionV3(include_top=True,weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-3].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

In [90]:
image_model.summary()


                 
__________________________________________________________________________________________________
batch_normalization_909 (BatchN (None, 17, 17, 192)  576         conv2d_909[0][0]                 
__________________________________________________________________________________________________
batch_normalization_914 (BatchN (None, 17, 17, 192)  576         conv2d_914[0][0]                 
__________________________________________________________________________________________________
batch_normalization_915 (BatchN (None, 17, 17, 192)  576         conv2d_915[0][0]                 
__________________________________________________________________________________________________
activation_906 (Activation)     (None, 17, 17, 192)  0           batch_normalization_906[0][0]    
__________________________________________________________________________________________________
activation_909 (Activation)     (None, 17, 17, 192)  0           batch_normalization_909[0]

In [91]:
image_features_extract_model.summary()

][0]             
__________________________________________________________________________________________________
conv2d_915 (Conv2D)             (None, 17, 17, 192)  147456      average_pooling2d_87[0][0]       
__________________________________________________________________________________________________
batch_normalization_906 (BatchN (None, 17, 17, 192)  576         conv2d_906[0][0]                 
__________________________________________________________________________________________________
batch_normalization_909 (BatchN (None, 17, 17, 192)  576         conv2d_909[0][0]                 
__________________________________________________________________________________________________
batch_normalization_914 (BatchN (None, 17, 17, 192)  576         conv2d_914[0][0]                 
__________________________________________________________________________________________________
batch_normalization_915 (BatchN (None, 17, 17, 192)  576         conv2d_915[0][0]          

In [95]:
# each folder contains three subfolders in accordance with the number of classes
train_dir = '/home/jhonat/Documents/tcc/dataset/train'
validation_dir = '/home/jhonat/Documents/tcc/dataset/test'
 
# the number of images for train and test is divided into 80:20 ratio
nTrain = 240
nVal = 50


In [96]:
# load the normalized images
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# define the batch size
batch_size = 20

# the defined shape is equal to the network output tensor shape
train_features = np.zeros(shape=(nTrain, 8, 8, 2048))
train_labels = np.zeros(shape=(nTrain,3))

# generate batches of train images and labels
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

Found 16200 images belonging to 3 classes.


In [97]:
# iterate through the batches of train images and labels
for i, (inputs_batch, labels_batch) in enumerate(train_generator):
    if i * batch_size >= nTrain:
        break    
    # pass the images through the network
    features_batch = image_features_extract_model.predict(inputs_batch)
    train_features[i * batch_size : (i + 1) * batch_size] = features_batch
    train_labels[i * batch_size : (i + 1) * batch_size] = labels_batch

# reshape train_features into vector        
train_features_vec = np.reshape(train_features, (nTrain, 8 * 8 * 2048))
print("Train features: {}".format(train_features_vec.shape))

Train features: (240, 131072)


In [107]:
frames_num = 15
count = 0
join_transfer = []
for i in range(int(len(train_features_vec)/frames_num)):
    inc = count+frames_num
    join_transfer.append([train_features_vec[count:inc], train_labels[count]])
    count = inc